In [46]:
import pandas as pd
import numpy as np
from tools import load_data, preprocess_data
from pt_preprocessing.imputation import fill_na_mode

In [35]:
df = load_data('clean_ufo_sightings.csv', False)

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

In [37]:
shape_encoded = OneHotEncoder().fit_transform(df['shape'].astype(object)[:, np.newaxis])
new_df = pd.concat([df.duration_sec, pd.DataFrame(shape_encoded.toarray())], axis=1)

In [38]:
from sklearn.model_selection import train_test_split

target_columns = ['latitude', 'longitude']
feature_columns = ['country', 'state', 'duration_sec']
X_train, X_test, y_train, y_test = train_test_split(new_df, df.loc[:, target_columns], test_size=0.2)

In [39]:
from sklearn.ensemble import RandomForestRegressor

In [40]:
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [41]:
y_pred = rf.predict(X_test)

In [43]:
from sklearn.metrics import r2_score

In [45]:
r2_score(y_test, y_pred) # slightly worse then predicting expected value for latitude and longitude

-0.017812375954396997

In [49]:
df.country = fill_na_mode(df.country)

In [52]:
country_encoded = OneHotEncoder().fit_transform(df.country[:, np.newaxis])

In [54]:
new_df = pd.concat([new_df, pd.DataFrame(country_encoded.toarray())], axis=1)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(new_df, df.loc[:, target_columns], test_size=0.2)

In [60]:
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
r2_score(y_test, y_pred)  # better!

0.35936674095806137